# CcGAN Loaded

This notebook is used for loading CcGAN and generating images.

## Step 1 - Import and Settings

Import others' libraries.

In [22]:
import numpy as np
import os
import random
import torch.backends.cudnn as cudnn
from torchvision.utils import save_image

Import our own libraries.

In [23]:
from models import *

Set hyperparameters.

In [24]:
MODEL_CcGAN_PATH = "/home/ubuntu/Desktop/Ra/models/CcGAN/ckpt_SAGAN_niters_30000_nDsteps_2_seed_2020_soft_0.11059499831726932_128.44444444444406.pth"
MODEL_EMBED_PATH = "/home/ubuntu/Desktop/Ra/models/embed/ResNet34_embed_seed_2020.pth"
MODEL_Y2H_PATH = "/home/ubuntu/Desktop/Ra/models/embed/ResNet34_y2h_seed_2020.pth"
IMAGE_DIR = "/home/ubuntu/Desktop/Ra/images/Ra_128_CcGAN_generate"
DIM_GAN = 128
NET_EMBED = "ResNet34_embed"
DIM_EMBED = 128
IMG_SIZE = 128
MIN_LABEL = 1.5
MAX_LABEL = 4.9
SEED = 2020

Settings.

In [25]:
# Create dir
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR)

# Set random seed
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
cudnn.benchmark = False
np.random.seed(SEED)

## Step 2 - Load Pretrained Embedding Models

In this step we load `net_embed` and `net_y2h` models to CPU.

In [26]:
if NET_EMBED == "ResNet18_embed":
    net_embed = ResNet18_embed(dim_embed=DIM_EMBED)
elif NET_EMBED == "ResNet34_embed":
    net_embed = ResNet34_embed(dim_embed=DIM_EMBED)
elif NET_EMBED == "ResNet50_embed":
    net_embed = ResNet50_embed(dim_embed=DIM_EMBED)
net_embed = net_embed.cuda()
net_embed = nn.DataParallel(net_embed)

net_y2h = model_y2h(dim_embed=DIM_EMBED)
net_y2h = net_y2h.cuda()
net_y2h = nn.DataParallel(net_y2h)

## (1). Load net_embed first: x2h+h2y
checkpoint = torch.load(MODEL_EMBED_PATH)
net_embed.load_state_dict(checkpoint['net_state_dict'])

## (2). Load y2h
checkpoint = torch.load(MODEL_Y2H_PATH)
net_y2h.load_state_dict(checkpoint['net_state_dict'])

net_embed.eval()
net_h2y = net_embed.module.h2y
net_y2h.eval()

DataParallel(
  (module): model_y2h(
    (main): Sequential(
      (0): Linear(in_features=1, out_features=128, bias=True)
      (1): GroupNorm(8, 128, eps=1e-05, affine=True)
      (2): ReLU()
      (3): Linear(in_features=128, out_features=128, bias=True)
      (4): GroupNorm(8, 128, eps=1e-05, affine=True)
      (5): ReLU()
      (6): Linear(in_features=128, out_features=128, bias=True)
      (7): GroupNorm(8, 128, eps=1e-05, affine=True)
      (8): ReLU()
      (9): Linear(in_features=128, out_features=128, bias=True)
      (10): GroupNorm(8, 128, eps=1e-05, affine=True)
      (11): ReLU()
      (12): Linear(in_features=128, out_features=128, bias=True)
      (13): ReLU()
    )
  )
)

## Step 3 - Load CcGAN

This step loads `netG` from `.pth` file, and switch it to evaluation mode.

In [27]:
checkpoint = torch.load(MODEL_CcGAN_PATH)
netG = CcGAN_SAGAN_Generator(dim_z=DIM_GAN, dim_embed=DIM_EMBED).cuda()
netG = nn.DataParallel(netG)
netG.load_state_dict(checkpoint['netG_state_dict'])
netG.eval()

DataParallel(
  (module): CcGAN_SAGAN_Generator(
    (snlinear0): Linear(in_features=128, out_features=16384, bias=True)
    (block1): GenBlock(
      (cond_bn1): ConditionalBatchNorm2d(
        (bn): BatchNorm2d(1024, eps=1e-05, momentum=0.001, affine=False, track_running_stats=True)
        (embed_gamma): Linear(in_features=128, out_features=1024, bias=False)
        (embed_beta): Linear(in_features=128, out_features=1024, bias=False)
      )
      (relu): ReLU(inplace=True)
      (snconv2d1): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cond_bn2): ConditionalBatchNorm2d(
        (bn): BatchNorm2d(1024, eps=1e-05, momentum=0.001, affine=False, track_running_stats=True)
        (embed_gamma): Linear(in_features=128, out_features=1024, bias=False)
        (embed_beta): Linear(in_features=128, out_features=1024, bias=False)
      )
      (snconv2d2): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (snconv2d0): Conv2d(1024, 1024

## Step 4 - Generate Images

We need to write `labels_to_generate` — a `numpy.ndarray` of all the labels (raw) with which we want to generate images.

In [28]:
# Labels to generate
labels_to_generate_raw = np.linspace(MIN_LABEL, MAX_LABEL, num=20).astype(float)

# Calculate
labels_to_generate = (labels_to_generate_raw - MIN_LABEL) / (MAX_LABEL - MIN_LABEL)
z = torch.randn(labels_to_generate.shape[0], IMG_SIZE, dtype=torch.float).cuda()
y = torch.from_numpy(labels_to_generate).type(torch.float).reshape(-1,1).cuda()

# Generate and save fake images
batch_fake_images = netG(z, net_y2h(y))
batch_fake_images = batch_fake_images.detach().cpu()
for i in range(batch_fake_images.shape[0]):
    fake_image = batch_fake_images[i].unsqueeze(0)
    save_image(fake_image.data, os.path.join(IMAGE_DIR, f"{i+1}_{labels_to_generate_raw[i]:.3f}.png"), nrow=1, normalize=True)
print(f"Generated {len(batch_fake_images)} images and saved to {IMAGE_DIR}")

Generated 20 images and saved to /home/ubuntu/Desktop/Ra/images/Ra_128_CcGAN_generate
